In [1]:
import numpy  as np

In [291]:
with open('data/input_13.txt') as fh:
    file_input = fh.read()

In [13]:
test_input = r"""/->-\        
|   |  /----\
| /-+--+-\  |
| | |  | v  |
\-+-/  \-+--/
  \------/   """

In [331]:
test_input2 = r"""/>-<\  
|   |  
| /<+-\
| | | v
\>+</ |
  |   ^
  \<->/"""

In [273]:
map_grid = {
    " ": 0,
    "|": 1,
    "v": 1,
    "^": 1,
    "-": 2,
    ">": 2,
    "<": 2,
    "/": 3,
    "\\": 4,
    "+": 5
}

map_dir = {
    ">" : 0,
    "v" : 1,
    "<" : 2,
    "^" : 3
}

In [274]:
def parse(line):
    grid = [map_grid[i] for i in line] 
    dirs = [map_dir[i] if i in map_dir else -1 for i in line]
    return grid, dirs

In [341]:
turn_left = lambda x: (x-1) % 4
turn_right= lambda x: (x+1) % 4

def advance(i, pos, mem, dirs, grid):
    x = pos['x'][i]
    y = pos['y'][i]
    
    dx, dy = {
        0: (1, 0),
        1: (0, 1),
        2: (-1, 0),
        3: (0, -1)        
    }[dirs[i]]
    
    x += dx
    y += dy

    #set new position
    pos['x'][i] = x
    pos['y'][i] = y
    
    field = grid[y, x]
    if field == 0:
        raise ValueError('Cart {} derailed!'.format(i))

    direction = dirs[i]
    if field >= 3:
        if field == map_grid['/']:
            if (direction == map_dir['^']) or (direction == map_dir['v']):
                direction = turn_right(direction)
            else:
                direction = turn_left(direction)
        
        if field == map_grid['\\']:
            if (direction == map_dir['^']) or (direction == map_dir['v']):
                direction = turn_left(direction)
            else:
                direction = turn_right(direction)            
        if field == map_grid['+']:
            if mem[i] == 0:
                direction = turn_left(direction)
            elif mem[i] == 2:
                direction = turn_right(direction)
            mem[i] = (mem[i] + 1) % 3
        dirs[i] = direction
        
    return (pos, mem, dirs, grid)

def visualize(grid, pos, dirs, crash, write_output=False):
    ny, nx = grid.shape
    lines = []
    for y in range(ny):        
        line = [{
            0: " ",
            1: "|",
            2: "-",
            3: "/",
            4: "\\",
            5: "+",
        }[i] for i in grid[y, :]]
        lines.append(line)
    for i, p in enumerate(pos):
        lines[p['y']][p['x']] = {
            0: ">",
            1: "v",
            2: "<",
            3: "^",
        }[dirs[i]]
    for p in crash[1]:
        lines[p['y']][p['x']] = 'X'
    if write_output is not None:
        with open('ascii/image_{:03d}.txt'.format(write_output), 'w') as fh:
            fh.write('text 15,15 "')
            for line in lines:
                fh.write("".join(line) + "\n")
            fh.write('    "')
    else:
        for line in lines:
            print ("".join(line))
            

def tick(pos, mem, dirs, grid):
    order = pos.argsort(order=['y', 'x'])
    crash = []
    for idx in order:
        pos, mem, dirs, grid = advance(idx, pos, mem, dirs, grid)    
#         visualize(grid, pos, dirs)
        for i, p in enumerate(pos):
            if i == idx:
                continue
            if p == pos[idx]:
                crash.append(idx)
                crash.append(i)
    ret = (crash, pos[crash])
    return ret
    

Part A+B

In [342]:
# inp = test_input.split('\n')
# inp = test_input2.split('\n')
inp = file_input.split('\n')[:-1]

data = np.array(list(map(parse, inp)))
grid = data[:,0,:].copy()
dirs = data[:,1,:].copy()
initial_pos = data[:,1,:].copy()
ny, nx = grid.shape

pos = np.array(np.where(initial_pos >= 0))
dirs = dirs[np.where(initial_pos >= 0)]
pos = np.rec.fromarrays(pos, dtype = [('y', '<i4'), ('x', '<i4')])
ncarts = len(pos)
mem = np.zeros(ncarts)

#visualize(grid, pos, dirs)
print("start")
i = 0
while True:
    #print('tick {}'.format(i))
    crash = tick(pos, mem, dirs, grid)
    visualize(grid, pos, dirs, crash, write_output=i)
    i += 1
    if crash[0]:
        # print (i, crash)
        for p in crash[1]:
            print ("{}: Crash at: {},{}".format(i, p['x'], p['y']))        
        mem = np.delete(mem, crash[0])
        dirs = np.delete(dirs, crash[0])
        pos = np.delete(pos, crash[0])
        print ("{} carts left".format(len(pos)))
    if len(pos) <= 1:
        print("last cart position {},{}".format(pos[0]['x'], pos[0]['y']))
        break
    if i > 400:
        print("aborted")
        break

start
165: Crash at: 39,52
165: Crash at: 38,52
15 carts left
384: Crash at: 34,82
384: Crash at: 34,82
13 carts left
396: Crash at: 117,124
396: Crash at: 117,124
11 carts left
aborted
